## Module Loading

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from IPython.core.pylabtools import figsize
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string
from string import punctuation
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.utils.extmath import randomized_svd
from sklearn.metrics import auc, roc_curve, plot_roc_curve, plot_confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn import datasets, metrics, model_selection, svm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from joblib import Memory
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans,  AgglomerativeClustering
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score
from sklearn.metrics import confusion_matrix 
from scipy.optimize import linear_sum_assignment
!pip install umap-learn
import umap
!pip install hdbscan
import hdbscan
import pickle
import bz2
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.model_selection import train_test_split
from plotmat import plot_mat

In [ ]:
# helper code
!pip install torch
!pip install torchvision
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import requests
import os
import tarfile
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import TransformerMixin

## Question 1

Report the dimensions of the TF-IDF matrix you obtain.

Ans:

the dimensions of the TF-IDF matrix is (7882, 23522).

In [ ]:
categories = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware',
              'comp.sys.mac.hardware','rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']
dataset = fetch_20newsgroups(subset = 'all', categories = categories, shuffle = True, random_state = 0,remove=('headers','footers'))

vec = CountVectorizer(stop_words='english', min_df=3)
tfidf = TfidfTransformer()

X_vec = vec.fit_transform(dataset.data)
X_tfidf = tfidf.fit_transform(X_vec)
print("TFIDX dataset shape:", X_tfidf.shape)

## Question 2

Report the contingency table of your clustering result. You may use
the provided plotmat.py to visualize the matrix. Does the contingency matrix have to be square-shaped?


Ans:

The contingency table is 

|3232|671|
|---|---|
|54|3925|

The contingency matrix doesn't have to be square-shaped but it's usually square-shaped. It has normally same rows and columns because the reference data and the map should have same dimensions (categories). However, it's not necessary, a category can exist in the reference data but doesn't exist in the map, and vice versa.

In [ ]:
# map_root = {"comp.graphics":0, "comp.os.ms-windows.misc":0, "comp.sys.ibm.pc.hardware":0, "comp.sys.mac.hardware":0,
#             "rec.autos":1, "rec.motorcycles":1, "rec.sport.baseball":1, "rec.sport.hockey":1}
# y_data = dataset.target.map(map_root)
y_data = []
for i in dataset.target:
  if i < 4:
    y_data.append(0)
  else:
    y_data.append(1)

In [ ]:
km = KMeans(n_clusters=2, init='k-means++', max_iter=2000, n_init=50, random_state=0)
km.fit(X_tfidf)

In [ ]:
plot_mat(contingency_matrix(y_data, km.labels_), size=(8, 6), xticklabels=['comp.','rec.'], yticklabels=['comp.','rec.'], pic_fname='Q2.png')

## Question 3

Report the 5 clustering measures explained in the introduction for Kmeans clustering.

Ans:

- Homogeneity: 0.5999
- Completeness: 0.6121
- V-measure: 0.6059
- Adjusted Rand-Index: 0.6659
- Adjusted Mutual Information Score: 0.6059

In [ ]:
def print_5_measure_scores(y_data, labels):
  print("Homogeneity: %0.4f" % homogeneity_score(y_data, labels))
  print("Completeness: %0.4f" % completeness_score(y_data, labels))
  print("V-measure: %0.4f" % v_measure_score(y_data, labels))
  print("Adjusted Rand-Index: %0.4f"% adjusted_rand_score(y_data, labels))
  print("Adjusted Mutual Information Score: %0.4f"% adjusted_mutual_info_score(y_data, labels))
print_5_measure_scores(y_data, km.labels_)

## Question 4

Report the plot of the percentage of variance that the top r principle
components retain v.s. r, for r = 1 to 1000.

In [ ]:
svd = TruncatedSVD(n_components=1000, random_state=0)
X_LSI = svd.fit_transform(X_tfidf)

ratios = np.cumsum(svd.explained_variance_ratio_)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(np.linspace(1, 1000, 1000), ratios, lw=2, linestyle='--')

ax.set_ylabel('cumulative sum of explained_variance_ratio')
ax.set_xlabel('principal components (r)')
plt.title('the percentage of variance that the top r principle components retain v.s. r')
plt.show()

## Question 5

Let $r$ be the dimension that we want to reduce the data to (i.e. n components).
Try $r$ = 1, 10, 20, 50, 100, 300, and plot the 5 measure scores v.s. $r$ for both SVD and NMF.

In [ ]:
rs5 = [1, 10, 20, 50, 100, 300]
# svd
svd_homo = []
svd_comp = []
svd_vmes = []
svd_ranI = []
svd_muts = []
# nmf
nmf_homo = []
nmf_comp = []
nmf_vmes = []
nmf_ranI = []
nmf_muts = []
kmm = KMeans(n_clusters=2, init='k-means++', max_iter=2000, n_init=50, random_state=0)
for i in range(len(rs5)):
  print("Try r =", rs5[i], "...")
  # SVD
  svd_tmp = TruncatedSVD(n_components=rs5[i], random_state=0)
  X_svd = svd_tmp.fit_transform(X_tfidf)
  km_svd = kmm.fit(X_svd)
  svd_homo.append(homogeneity_score(y_data, km_svd.labels_))
  svd_comp.append(completeness_score(y_data, km_svd.labels_))
  svd_vmes.append(v_measure_score(y_data, km_svd.labels_))
  svd_ranI.append(adjusted_rand_score(y_data, km_svd.labels_))
  svd_muts.append(adjusted_mutual_info_score(y_data, km_svd.labels_))
  # NMF
  nmf_tmp = NMF(n_components=rs5[i], init='random', random_state=0)
  X_nmf = nmf_tmp.fit_transform(X_tfidf)
  km_nmf = kmm.fit(X_nmf)
  nmf_homo.append(homogeneity_score(y_data, km_nmf.labels_))
  nmf_comp.append(completeness_score(y_data, km_nmf.labels_))
  nmf_vmes.append(v_measure_score(y_data, km_nmf.labels_))
  nmf_ranI.append(adjusted_rand_score(y_data, km_nmf.labels_))
  nmf_muts.append(adjusted_mutual_info_score(y_data, km_nmf.labels_))
print('Done')


In [ ]:
def print_bar_scores_result(width, rs, svd_score, nmf_score, score_name, title):
  fig = plt.figure()
  ax = fig.add_subplot(1, 1, 1)
  # ax.plot(rs, svd_homo, color='r', label="SVD")
  # ax.plot(rs, nmf_homo, color='b', label="NMF")
  ax.bar(np.arange(len(rs)) - width/2, svd_score, width, label="SVD")
  ax.bar(np.arange(len(rs)) + width/2, nmf_score, width, label="NMF")
  ax.set_xticks(np.arange(len(rs)))
  ax.set_xticklabels(rs)

  ax.legend()
  ax.set_ylabel(score_name)
  ax.set_xlabel('r')
  plt.title(title)
  plt.show()

In [ ]:
print_bar_scores_result(0.3, rs5, svd_homo, nmf_homo, 'Homogeneity', 'Homogenity Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs5, svd_comp, nmf_comp, 'Completeness', 'Completeness Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs5, svd_vmes, nmf_vmes, 'V-measure', 'V-measure Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs5, svd_ranI, nmf_ranI, 'Adjusted Rand-Index', 'Adjusted Rand-Index for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs5, svd_muts, nmf_muts, 'Adjusted Mutual Information Score', 'Adjusted Mutual Information Score for different r [SVD vs. NMF]')

Report a good choice of $r$ for SVD and NMF respectively.

Ans:

From the graph above, we can find that a good choice of $r$ for SVD could be $r = 50$, a good choice of $r$ for NMF could be $r = 10$.

Because we can find that there are no significant increase for SVD after $r = 50$. And as $r$ increases, we need to realize that the Euclidean distances in K-means we use will converge to a constant value, such that K-means doesn't perform well in high-dimensional prediction. Considering to this, choosing $r = 50$ is good enough. On the other hand, NMF performs obviously best when $r = 10$.

## Question 6

How do you explain the non-monotonic behavior of the measures as $r$
increases?

Ans:

As $r$ increases, the performance of the measures doesn't increase correspondingly. It's because that as $r$ increases, it indicates that the target matrix behind it will become complicated and high-dimensional. In which restrained the performance of K-means as we mentioned before. The Euclidean distances in K-means will converge to a constant value between the sample points. The clustering method can't find centroids for different sample points so that its performance becomes poor.

On the other hand, we can find that SVD maintain the similar score performance even after $r$ keeps increasing, however, NMF doesn't perform well after $r$ is larger than $10$. It might because of the fact that SVD is a much more deterministic method than NMF. As we discussed in Project 1, SVD is a more insightful method and is able to interpret high-dimensional data rather than NMF did. Since NMF only uses the positive entries in the reduced matrix and makes assumption about the missing values. SVD doesn't assume anything about the value. Therefore, it's less restricted and performs better in high-dimensional data.

## Question 7

Are these measures on average better than those computed in Question
3?

Ans:

No, even though we minus the outlier data ($r = 1$), the average performance of these measures is still worse than those computed in Question 3.

In [ ]:
print("Measure scores computed in Question 3:")
print_5_measure_scores(y_data, km.labels_)

print("\nAverage measure scores for SVD:")
print("Homogeneity: %0.4f" % (sum(svd_homo) / len(svd_homo)))
print("Completeness: %0.4f" % (sum(svd_comp) / len(svd_comp)))
print("V-measure: %0.4f" % (sum(svd_vmes) / len(svd_vmes)))
print("Adjusted Rand-Index: %0.4f" % (sum(svd_ranI) / len(svd_ranI)))
print("Adjusted Mutual Information Score: %0.4f" % (sum(svd_muts) / len(svd_muts)))

print("\nAverage measure scores for NMF:")
print("Homogeneity: %0.4f" % (sum(nmf_homo) / len(nmf_homo)))
print("Completeness: %0.4f" % (sum(nmf_comp) / len(nmf_comp)))
print("V-measure: %0.4f" % (sum(nmf_vmes) / len(nmf_vmes)))
print("Adjusted Rand-Index: %0.4f" % (sum(nmf_ranI) / len(nmf_ranI)))
print("Adjusted Mutual Information Score: %0.4f" % (sum(nmf_muts) / len(nmf_muts)))

print("\nAverage measure scores for SVD (not including r=1):")
print("Homogeneity: %0.4f" % ( (sum(svd_homo) - svd_homo[0]) / (len(svd_homo) - 1)))
print("Completeness: %0.4f" % ( (sum(svd_comp) - svd_comp[0]) / (len(svd_comp) - 1)))
print("V-measure: %0.4f" % ( (sum(svd_vmes) - svd_vmes[0]) / (len(svd_vmes) - 1)))
print("Adjusted Rand-Index: %0.4f" % ( (sum(svd_ranI) - svd_ranI[0]) / (len(svd_ranI) - 1)))
print("Adjusted Mutual Information Score: %0.4f" % ( (sum(svd_muts) - svd_ranI[0]) / (len(svd_muts) - 1)))

## Question 8

Visualize the clustering results for:

- SVD with your optimal choice of r for K-Means clustering;

- NMF with your choice of r for K-Means clustering.

In [ ]:
# best parameter of SVD and NMF
best_r_SVD = 50
best_r_NMF = 10
kmm = KMeans(n_clusters=2, init='k-means++', max_iter=2000, n_init=50, random_state=0)
# best SVD
best_svd = TruncatedSVD(n_components=best_r_SVD, random_state=0)
best_X_svd = best_svd.fit_transform(X_tfidf)
y_pred_best_svd = kmm.fit_predict(best_X_svd)
# best NMF
best_nmf = NMF(n_components=best_r_NMF, init='random', random_state=0)
best_X_nmf = best_nmf.fit_transform(X_tfidf)
y_pred_best_nmf = kmm.fit_predict(best_X_nmf)

In [ ]:
plt.scatter(best_X_svd[:,0], best_X_svd[:,1], c=y_data)
plt.title("SVD with r=50 for clustering with real labels")
plt.show()
plt.scatter(best_X_svd[:,0], best_X_svd[:,1], c=y_pred_best_svd)
plt.title("SVD with r=50 for clustering with K-means predicted labels")
plt.show()

In [ ]:
plt.scatter(best_X_nmf[:,0], best_X_nmf[:,1], c=y_data)
plt.title("NMF with r=10 for clustering with real labels")
plt.show()
plt.scatter(best_X_nmf[:,0], best_X_nmf[:,1], c=y_pred_best_nmf)
plt.title("NMF with r=10 for clustering with K-means predicted labels")
plt.show()

## Question 9

What do you observe in the visualization? How are the data points of the
two classes distributed? Is distribution of the data ideal for K-Means clustering?

Ans:

1. From the visualization, we can see that there are no spherical distributions on the plots for both SVD or NMF. there are many overlapping points among the two clusters which makes the boundary of two clusters hard to define.
2. The data points in the SVD are distributed more ideally than NMF did. Its shape also looks more like a sphere comparing to NMF one. On the other hand, we can analyze the performances from their homogeneity scores. SVD gets a higher score than NMF did but it's still not great enough.
3. In conclusion, the distribution of the data is not ideal for K-Means clustering. K-Means clustering assumes spherical shapes of clusters but we can not find a similar shape no matter in SVD or NMF here.

## Question 10

Load documents with the same configuration as in Question 1, but for
ALL 20 categories.


There is a mismatch between cluster labels and class labels. For example, the cluster #3 may correspond to the class #8. As a result, the high-value entries of the 20 × 20 contingency matrix can be scattered around, making it messy to inspect, even if the clustering result is not bad.

In [ ]:
dataset_all = fetch_20newsgroups(subset = 'all', shuffle = True, random_state = 0,remove=('headers','footers'))

vec = CountVectorizer(stop_words='english', min_df=3)
tfidf = TfidfTransformer()

X_a_vec = vec.fit_transform(dataset_all.data)
X_a_tfidf = tfidf.fit_transform(X_a_vec)
print("TFIDX dataset shape:", X_a_tfidf.shape)

In [ ]:
y_a_data = dataset_all.target

kma = KMeans(n_clusters=20, init='k-means++', max_iter=2000, n_init=50, random_state=0)
# kma.fit(X_a_tfidf)

Construct the TF-IDF matrix, reduce its dimensionality using BOTH NMF
and SVD (specify settings you choose and why).

Ans:

We choose $r = 20$ for SVD, $r = 10$ for NMF respectively. we will explain the reason in the following cells.

In [ ]:
rs10 = [1, 5, 10, 20, 50, 100]
# svd
a_svd_homo = []
a_svd_comp = []
a_svd_vmes = []
a_svd_ranI = []
a_svd_muts = []
# nmf
a_nmf_homo = []
a_nmf_comp = []
a_nmf_vmes = []
a_nmf_ranI = []
a_nmf_muts = []
for i in range(len(rs10)):
  print("Try r =", rs10[i], "...")
  # SVD
  svd_tmp = TruncatedSVD(n_components=rs10[i], random_state=0)
  X_a_svd = svd_tmp.fit_transform(X_a_tfidf)
  kma_svd = kma.fit(X_a_svd)
  a_svd_homo.append(homogeneity_score(y_a_data, kma_svd.labels_))
  a_svd_comp.append(completeness_score(y_a_data, kma_svd.labels_))
  a_svd_vmes.append(v_measure_score(y_a_data, kma_svd.labels_))
  a_svd_ranI.append(adjusted_rand_score(y_a_data, kma_svd.labels_))
  a_svd_muts.append(adjusted_mutual_info_score(y_a_data, kma_svd.labels_))
  # NMF
  nmf_tmp = NMF(n_components=rs10[i], init='random', random_state=0)
  X_a_nmf = nmf_tmp.fit_transform(X_a_tfidf)
  kma_nmf = kma.fit(X_a_nmf)
  a_nmf_homo.append(homogeneity_score(y_a_data, kma_nmf.labels_))
  a_nmf_comp.append(completeness_score(y_a_data, kma_nmf.labels_))
  a_nmf_vmes.append(v_measure_score(y_a_data, kma_nmf.labels_))
  a_nmf_ranI.append(adjusted_rand_score(y_a_data, kma_nmf.labels_))
  a_nmf_muts.append(adjusted_mutual_info_score(y_a_data, kma_nmf.labels_))
print('Done')

In [ ]:
print_bar_scores_result(0.3, rs10, a_svd_homo, a_nmf_homo, 'Homogeneity', 'Homogenity Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs10, a_svd_comp, a_nmf_comp, 'Completeness', 'Completeness Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs10, a_svd_vmes, a_nmf_vmes, 'V-measure', 'V-measure Score for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs10, a_svd_ranI, a_nmf_ranI, 'Adjusted Rand-Index', 'Adjusted Rand-Index for different r [SVD vs. NMF]')
print_bar_scores_result(0.3, rs10, a_svd_muts, a_nmf_muts, 'Adjusted Mutual Information Score', 'Adjusted Mutual Information Score for different r [SVD vs. NMF]')

In [ ]:
avg_svd_homo = sum(a_svd_homo) / len(a_svd_homo)
avg_svd_comp = sum(a_svd_comp) / len(a_svd_comp)
avg_svd_vmes = sum(a_svd_vmes) / len(a_svd_vmes)
avg_svd_ranI = sum(a_svd_ranI) / len(a_svd_ranI)
avg_svd_muts = sum(a_svd_muts) / len(a_svd_muts)

avg_nmf_homo = sum(a_nmf_homo) / len(a_nmf_homo)
avg_nmf_comp = sum(a_nmf_comp) / len(a_nmf_comp)
avg_nmf_vmes = sum(a_nmf_vmes) / len(a_nmf_vmes)
avg_nmf_ranI = sum(a_nmf_ranI) / len(a_nmf_ranI)
avg_nmf_muts = sum(a_nmf_muts) / len(a_nmf_muts)

avg_svd = []
avg_nmf = []
for i in range(len(rs10)):
  tmp_svd = 0
  tmp_svd += a_svd_homo[i] / avg_svd_homo
  tmp_svd += a_svd_comp[i] / avg_svd_comp
  tmp_svd += a_svd_vmes[i] / avg_svd_vmes
  tmp_svd += a_svd_ranI[i] / avg_svd_ranI
  tmp_svd += a_svd_muts[i] / avg_svd_muts
  avg_svd.append(tmp_svd / 5)
  tmp_nmf = 0
  tmp_nmf += a_nmf_homo[i] / avg_nmf_homo
  tmp_nmf += a_nmf_comp[i] / avg_nmf_comp
  tmp_nmf += a_nmf_vmes[i] / avg_nmf_vmes
  tmp_nmf += a_nmf_ranI[i] / avg_nmf_ranI
  tmp_nmf += a_nmf_muts[i] / avg_nmf_muts
  avg_nmf.append(tmp_nmf / 5)
for i in range(len(avg_svd)):
  print("Average normalized scores for SVD when r =", rs10[i], ":", avg_svd[i])
print("")
for i in range(len(avg_nmf)):
  print("Average normalized scores for NMF when r =", rs10[i], ":", avg_nmf[i])

sorted_avg_svd = sorted(avg_svd, reverse=True)
sorted_avg_nmf = sorted(avg_nmf, reverse=True)

top3_svd = [avg_svd.index(v) for v in sorted_avg_svd[:3]]
top3_nmf = [avg_nmf.index(v) for v in sorted_avg_nmf[:3]]

print("\nTop 3 best values of r for SVD:")
for i in top3_svd:
  print("r =", rs10[i], ":", avg_svd[i])

print("\nTop 3 best values of r for NMF:")
for i in top3_nmf:
  print("r =", rs10[i], ":", avg_nmf[i])

We computed the 5 measure scores, normalized it to sum up and get average. As you can see, we obtained the top 3 best values of r for SVD and NMF respectively.

Based on above result, we choose $r = 20$ for SVD, $r = 10$ for NMF as setting. 

As $r$ increases to 100 for SVD, there is no corresponding significant increasing in the normalized scores for it. Therefore, choosing $r = 20$ is good enough.

Choosing $r = 10$ for NMF is easy here, since it's lower than $20$ but get a higher normalized average score. 

Visualize the contingency matrix and report the five clustering metrics (DO BOTH
NMF AND SVD).

In [ ]:
best_a_svd = 20
# SVD
best_svd = TruncatedSVD(n_components=best_a_svd, random_state=0)
best_X_a_svd = best_svd.fit_transform(X_a_tfidf)
best_kma_svd = kma.fit(best_X_a_svd)
cm = confusion_matrix(dataset_all.target, kma.labels_)
rows, cols = linear_sum_assignment(cm, maximize=True)
plot_mat(cm[rows[:, np.newaxis], cols], xticklabels=cols, yticklabels=rows, title='SVD with best r (r = 20)', size=(15,15))

In [ ]:
best_a_nmf = 10
# NMF
best_nmf = NMF(n_components=best_a_nmf, random_state=0)
best_X_a_nmf = best_nmf.fit_transform(X_a_tfidf)
best_kma_nmf = kma.fit(best_X_a_nmf)
cm = confusion_matrix(dataset_all.target, kma.labels_)
rows, cols = linear_sum_assignment(cm, maximize=True)
plot_mat(cm[rows[:, np.newaxis], cols], xticklabels=cols, yticklabels=rows, title='NMF with best r (r = 10)', size=(15,15))

## Question 11

Reduce the dimension of your dataset with UMAP. Consider the following
settings: n components = [5, 20, 200], metric = ”cosine” vs. ”euclidean”. If ”cosine” metric fails, please look at the FAQ at the end of this spec.

Report the permuted contingency matrix and the five clustering evaluation metrics for the different combinations (6 combinations).

In [ ]:
rs11 = [5, 20, 200]
# metrics = ['cosine', 'euclidean']
# UMAP cosine
umap_cos_homo = []
umap_cos_comp = []
umap_cos_vmes = []
umap_cos_ranI = []
umap_cos_muts = []
# UMAP euclidean
umap_euc_homo = []
umap_euc_comp = []
umap_euc_vmes = []
umap_euc_ranI = []
umap_euc_muts = []

kma = KMeans(n_clusters=20, init='k-means++', max_iter=2000, n_init=50, random_state=0)

for i in range(len(rs11)):
  print("\nTry r =", rs11[i], "...")
  # cosine
  print("Cosine UMAP when r =", rs11[i], ":")
  tmp_umap_cos = umap.UMAP(n_components=rs11[i], metric='cosine')
  X_umap_cos = tmp_umap_cos.fit_transform(X_a_tfidf)
  kma_cos = kma.fit(X_umap_cos)
  umap_cos_homo.append(homogeneity_score(y_a_data, kma_cos.labels_))
  umap_cos_comp.append(completeness_score(y_a_data, kma_cos.labels_))
  umap_cos_vmes.append(v_measure_score(y_a_data, kma_cos.labels_))
  umap_cos_ranI.append(adjusted_rand_score(y_a_data, kma_cos.labels_))
  umap_cos_muts.append(adjusted_mutual_info_score(y_a_data, kma_cos.labels_))
  # print permuted contingency matrix
  cm = confusion_matrix(y_a_data, kma.labels_)
  rows, cols = linear_sum_assignment(cm, maximize=True)
  plot_mat(cm[rows[:, np.newaxis], cols], xticklabels=cols, yticklabels=rows, title='cosine UMAP with r=' + str(rs11[i]), size=(15,15))
  # print the five clustering evaluation metrics
  print_5_measure_scores(y_a_data, kma_cos.labels_)
  # euclidean
  print("\nEuclidean UMAP when r =", rs11[i], ":")
  tmp_umap_euc = umap.UMAP(n_components=rs11[i], metric='euclidean')
  X_umap_euc = tmp_umap_cos.fit_transform(X_a_tfidf)
  kma_euc = kma.fit(X_umap_euc)
  umap_euc_homo.append(homogeneity_score(y_a_data, kma_euc.labels_))
  umap_euc_comp.append(completeness_score(y_a_data, kma_euc.labels_))
  umap_euc_vmes.append(v_measure_score(y_a_data, kma_euc.labels_))
  umap_euc_ranI.append(adjusted_rand_score(y_a_data, kma_euc.labels_))
  umap_euc_muts.append(adjusted_mutual_info_score(y_a_data, kma_euc.labels_))
  # print permuted contingency matrix
  cm = confusion_matrix(y_a_data, kma.labels_)
  rows, cols = linear_sum_assignment(cm, maximize=True)
  plot_mat(cm[rows[:, np.newaxis], cols], xticklabels=cols, yticklabels=rows, title='euclidean UMAP with r=' + str(rs11[i]), size=(15,15))
  # print the five clustering evaluation metrics
  print_5_measure_scores(y_a_data, kma_euc.labels_)
print('Done')

## Question 12

Analyze the contingency matrices. Which setting works best and why?

Ans:

From the contingency metrices we found above, we can see that there are no many differences between the euclidean UMAP with different $r$. Therefore, setting $r = 20$ is good enough for euclidean UMAP.

As of cosine UMAP, setting $r = 20$ gives us a better result. Setting $r$ equals to other numbers all gives more outliers or categories being distributed to other categories.



In [ ]:
avg_cos = []
avg_euc = []
for i in range(len(rs11)):
  tmp_cos = 0
  tmp_cos += umap_cos_homo[i] / (sum(umap_cos_homo) / len(umap_cos_homo))
  tmp_cos += umap_cos_comp[i] / (sum(umap_cos_comp) / len(umap_cos_comp))
  tmp_cos += umap_cos_vmes[i] / (sum(umap_cos_vmes) / len(umap_cos_vmes))
  tmp_cos += umap_cos_ranI[i] / (sum(umap_cos_ranI) / len(umap_cos_ranI))
  tmp_cos += umap_cos_muts[i] / (sum(umap_cos_muts) / len(umap_cos_muts))
  avg_cos.append(tmp_cos / 5)
  tmp_euc = 0
  tmp_euc += umap_euc_homo[i] / (sum(umap_euc_homo) / len(umap_euc_homo))
  tmp_euc += umap_euc_comp[i] / (sum(umap_euc_comp) / len(umap_euc_comp))
  tmp_euc += umap_euc_vmes[i] / (sum(umap_euc_vmes) / len(umap_euc_vmes))
  tmp_euc += umap_euc_ranI[i] / (sum(umap_euc_ranI) / len(umap_euc_ranI))
  tmp_euc += umap_euc_muts[i] / (sum(umap_euc_muts) / len(umap_euc_muts))
  avg_euc.append(tmp_euc / 5)
for i in range(len(avg_cos)):
  print("Average normalized scores for cosine UMAP when r =", rs11[i], ":", avg_cos[i])
print("")
for i in range(len(avg_euc)):
  print("Average normalized scores for euclidean UMAP when r =", rs11[i], ":", avg_euc[i])

best_r_cos = rs11[avg_cos.index(max(avg_cos))]
# best_r_cos = 5
best_r_euc = rs11[avg_euc.index(max(avg_euc))]
print("\nBest value of r for cosine UMAP:", best_r_cos, ", its average normalized scores:", avg_cos[rs11.index(best_r_cos)])
print("Best value of r for euclidean UMAP:", best_r_euc, ", its average normalized scores:", avg_euc[rs11.index(best_r_euc)])

What about for each metric choice?

Ans:

From the result above, we calculated the average normalized scores for each combination.

 It also indicates that there are no many differences between the euclidean UMAP with different $r$. Although the scores when $r = 200$ is a little bit better, choosing $r = 20$ is good enough considering to the calculation time.

On the other hand, a good choice of $r$ for cosine UMAP is $20$.

Comparing the scores of these two UMAP, cosine UMAP with $r = 20$ works best and we choose it for the following comparison.

## Question 13

So far, we have attempted K-Means clustering with 4 different representation
learning techniques (sparse TF-IDF representation, PCA-reduced, NMF-reduced, UMAP-reduced).

Compare and contrast the clustering results across the 4 choices, and suggest an approach that is best for the K-Means clustering task on the 20-class text data. Choose any choice of clustering metrics for your comparison.

Ans:

Comparin the clustering results across the 4 choices, we can see that the performance of UMAP-reduced is better than other 3 results. Therefore, we suggest **UMAP-reduced (cosine UMAP with $r = 20$)** is best for the K-Means clustering task on the 20-class text data.

In [ ]:
def print_scores(title_name, score_homo, score_comp, score_vmes, score_ranI, score_muts):
  print(title_name)
  print("Homogeneity: %0.4f" % score_homo)
  print("Completeness: %0.4f" % score_comp)
  print("V-measure: %0.4f" % score_vmes)
  print("Adjusted Rand-Index: %0.4f" % score_ranI)
  print("Adjusted Mutual Information Score: %0.4f \n" % score_muts)

In [ ]:
kmm_o = KMeans(n_clusters=20, init='k-means++', max_iter=2000, n_init=50, random_state=0)
kmm_o.fit(X_a_tfidf)

In [ ]:
print("scores for sparse TF-IDF representation:")
print_5_measure_scores(y_a_data, kmm_o.labels_)
print("")
br_svd_i = rs10.index(best_a_svd)
br_nmf_i = rs10.index(best_a_nmf)
br_cos_i = avg_cos.index(max(avg_cos))
print_scores("scores for SVD when r = " + str(best_a_svd) + ":", a_svd_homo[br_svd_i], a_svd_comp[br_svd_i], a_svd_vmes[br_svd_i], a_svd_ranI[br_svd_i], a_svd_muts[br_svd_i])
print_scores("scores for NMF when r = " + str(best_a_nmf) + ":", a_nmf_homo[br_nmf_i], a_nmf_comp[br_nmf_i], a_nmf_vmes[br_nmf_i], a_nmf_ranI[br_nmf_i], a_nmf_muts[br_nmf_i])
print_scores("scores for cosine UMAP when r = " + str(rs11[br_cos_i]) + ":", umap_cos_homo[br_cos_i], umap_cos_comp[br_cos_i], umap_cos_vmes[br_cos_i], umap_cos_ranI[br_cos_i], umap_euc_muts[br_cos_i])

## Question 14

Use UMAP to reduce the dimensionality properly, and perform Agglomerative clustering with n_clusters=20 . Compare the performance of “ward” and “single”
linkage criteria.

Report the five clustering evaluation metrics for each case.

Ans:

From the five clustering evaluation metrics indicate below, we can find that the performance of Agglomerative Clustering with single linkage criterion is much worse than ward linkage criterion one did. 

It might because of the using method behind these two criterions. The ward linkage criterion minimizes the variance of the clusters being merged. The single linkage criterion minimizes the distance between all observations of the two sets. Therefore, single linkage criterion is not robust enough to handle this high dimensional problem. It fails to deal with the noise or outlier data.

In [ ]:
best_umap = umap.UMAP(n_components=rs11[br_cos_i], metric='cosine')
X_umap = best_umap.fit_transform(X_a_tfidf)
agg_w = AgglomerativeClustering(n_clusters=20, linkage='ward').fit(X_umap)
agg_s = AgglomerativeClustering(n_clusters=20, linkage='single').fit(X_umap)

print("Agglomerative Clustering with ward linkage criterion:")
print_5_measure_scores(y_a_data, agg_w.labels_)
print("\nAgglomerative Clustering with single linkage criterion:")
print_5_measure_scores(y_a_data, agg_s.labels_)

## Question 15

Apply HDBSCAN on UMAP-transformed 20-category data.

Use min_cluster_size=100 .

Vary the min_cluster_size among 20, 100, 200 and report your findings in terms of the five clustering evaluation metrics - you will plot the best contingency matrix in the next question. Feel free to try modifying other parameters in HDBSCAN to get better performance.

Ans:

From the five clustering evaluation metrics we found, the best min_cluster_size to use here is $100$.

In [ ]:
cluster_sizes = [20, 100, 200]
hdbs_homo = []
hdbs_comp = []
hdbs_vmes = []
hdbs_ranI = []
hdbs_muts = []

for min_size in cluster_sizes:
  print("\nmin_cluster_size =", min_size, ":")
  y_pred_hdbs = hdbscan.HDBSCAN(min_cluster_size=min_size).fit_predict(X_umap)
  hdbs_homo.append(homogeneity_score(y_a_data, y_pred_hdbs))
  hdbs_comp.append(completeness_score(y_a_data, y_pred_hdbs))
  hdbs_vmes.append(v_measure_score(y_a_data, y_pred_hdbs))
  hdbs_ranI.append(adjusted_rand_score(y_a_data, y_pred_hdbs))
  hdbs_muts.append(adjusted_mutual_info_score(y_a_data, y_pred_hdbs))
  print_5_measure_scores(y_a_data, y_pred_hdbs)

In [ ]:
avg_hdbs = []
for i in range(len(cluster_sizes)):
  tmp_hdbs = 0
  tmp_hdbs += hdbs_homo[i] / (sum(hdbs_homo) / len(hdbs_homo))
  tmp_hdbs += hdbs_comp[i] / (sum(hdbs_comp) / len(hdbs_comp))
  tmp_hdbs += hdbs_vmes[i] / (sum(hdbs_vmes) / len(hdbs_vmes))
  tmp_hdbs += hdbs_ranI[i] / (sum(hdbs_ranI) / len(hdbs_ranI))
  tmp_hdbs += hdbs_muts[i] / (sum(hdbs_muts) / len(hdbs_muts))
  avg_hdbs.append(tmp_hdbs / 5)
for i in range(len(avg_hdbs)):
  print("Average normalized scores for HDBSCAN when min_cluster_size =", cluster_sizes[i], ":", avg_hdbs[i])

best_mcs = cluster_sizes[avg_hdbs.index(max(avg_hdbs))]
print("\nBest value of min_cluster_size for HDBSCAN:", best_mcs, ", its average normalized scores:", max(avg_hdbs))

## Question 16

Contingency matrix

Plot the contingency matrix for the best clustering model from Question 15.
How many clusters are given by the model?

Interpret the contingency matrix considering the answer to these questions.

Ans:

According to our finding, there are total 10 major clusters given by the model. Furthermore, you can find there are many categories being distributed to other categories as a cluster. 

It might because of the reason that density based clustering relies on having enough data to separate dense areas. 
In higher dimensional spaces this becomes more difficult, and hence requires more data. Which makes HDBSCAN hard to perform well in this high dimensional data.

What does “-1” mean for the clustering labels?

Ans:

The “-1” in the clustering  means noise or outlier sammples that can not been classfied into a cluster by the algorithms.


In [ ]:
# print contingency matrix
by_pred_hdbs = hdbscan.HDBSCAN(min_cluster_size=best_mcs).fit_predict(X_umap)
cm16 = confusion_matrix(y_a_data, by_pred_hdbs)
rows, cols = linear_sum_assignment(cm16, maximize=True)
plot_mat(cm16[rows[:, np.newaxis], cols], xticklabels=cols, yticklabels=rows, title='HDBSCAN with min_cluster_size=' + str(best_mcs), size=(15,15))

## Question 17

Based on your experiments, which dimensionality reduction technique and clustering methods worked best together for 20-class text data and why? Follow the table below. If UMAP takes too long to converge, consider running it once and saving the intermediate results in a pickle file.

|Module|Alternatives|Hyperparameters|
|---|---|---|
|Dimensionality Reduction|None|N/A|
|Dimensionality Reduction|SVD|r = [5, 20, 200]|
|Dimensionality Reduction|NMF|r = [5, 20, 200]|
|Dimensionality Reduction|UMAP|n_components = [5, 20, 200]|
|Clustering|K-Means|k = [10, 20, 50]|
|Clustering|Agglomerative Clustering|n_clusters = [20]|
|Clustering|HDBSCAN|min_cluster_size = [100, 200]|

Ans:

The combination of **UMAP using 'cosine' matrix and n_components setting to 5 plus K-Means using k = 20** works best together for 20-class text data. we will analysis the reason of it in the following cells.

In [ ]:
def pkl_save(path, data, filename):
  with open(path + filename, 'wb') as f:
    # compressed_file = bz2.BZ2File(f, 'w')
    pickle.dump(data, f)

In [ ]:
dataset_all = fetch_20newsgroups(subset = 'all', shuffle = True, random_state = 0, remove=('headers','footers'))
vec = CountVectorizer(stop_words='english', min_df=3)
tfidf = TfidfTransformer()
X_vec = vec.fit_transform(dataset_all.data)
X_tfidf = tfidf.fit_transform(X_vec)
y_data = dataset_all.target

In [ ]:
# modify this line to your local path
pwd = !pwd
path = str(pwd[0]) + '/pickle file/'

In [ ]:
# None
pkl_save(path, X_tfidf, 'none.pkl')

rs17 = [5, 20, 200]
for i in range(len(rs17)):
  print("\ntry r =", rs17[i], "...")
  # SVD
  print("using SVD ...")
  svd_tmp = TruncatedSVD(n_components=rs17[i], random_state=0)
  X_svd = svd_tmp.fit_transform(X_tfidf)
  pkl_save(path, X_svd, 'svd_' + str(rs17[i]) + '.pkl')

  # NMF
  print("using NMF ...")
  nmf_tmp = NMF(n_components=rs17[i], init='random', random_state=0)
  X_nmf = nmf_tmp.fit_transform(X_tfidf)
  pkl_save(path, X_nmf, 'nmf_' + str(rs17[i]) + '.pkl')

  # UMAP
  print("using UMAP ...")
  umap_tmp = umap.UMAP(n_components=rs17[i], metric='cosine')
  X_umap = umap_tmp.fit_transform(X_tfidf)
  pkl_save(path, X_umap, 'umap_' + str(rs17[i]) + '.pkl')


In [ ]:
def pkl_read(path, filename):
  with open(path + filename, 'rb') as f:
    # compressed_file = bz2.BZ2File(f, 'r')
    X_dr = pickle.load(f)
  return X_dr
DRs = ['none.pkl',
       'svd_5.pkl', 'svd_20.pkl', 'svd_200.pkl',
       'nmf_5.pkl', 'nmf_20.pkl', 'nmf_200.pkl',
       'umap_5.pkl', 'umap_20.pkl', 'umap_200.pkl']

In [ ]:
ks = [10, 20, 50]
km_scores = [ [0] * len(DRs) for i in range(3)]
for i in range(len(ks)):
  print("\nk =", ks[i], "...")
  kmeans_tmp = KMeans(n_clusters=ks[i], init='k-means++', max_iter=2000, n_init=50, random_state=0)
  for j in range(len(DRs)):
    print("Current dr:", DRs[j])
    # reading Dimensionality Reduction data
    X_dr = pkl_read(path, DRs[j])
    kmeans_fit = kmeans_tmp.fit(X_dr)
    # To increase efficiency, we calculate the average score of each combination instead.
    tmp_s = 0
    tmp_s += homogeneity_score(y_data, kmeans_fit.labels_)
    tmp_s += completeness_score(y_data, kmeans_fit.labels_)
    tmp_s += v_measure_score(y_data, kmeans_fit.labels_)
    tmp_s += adjusted_rand_score(y_data, kmeans_fit.labels_)
    tmp_s += adjusted_mutual_info_score(y_data, kmeans_fit.labels_)
    tmp_s /= 5
    km_scores[i][j] = tmp_s
    print("Kmeans when k =", ks[i], ", average score:", tmp_s)

In [ ]:
agg_n = 20
agg_scores = [ [0] * len(DRs)]
for j in range(len(DRs)):
  print("Current dr:", DRs[j])
  # it takes over 1hr to finish the none.pkl, we save the result that we have found before in advance here
  if (DRs[j] == 'none.pkl'):
    print("Agglomerative Clustering when n_clusters =", agg_n, ", average score:", 0.3437828455142916)
    agg_scores[0][j] = 0.3437828455142916
    continue
  # reading Dimensionality Reduction data
  X_dr = pkl_read(path, DRs[j])
  # if (DRs[j] == 'none.pkl'):
  #   X_dr = X_dr.toarray()
  agg_tmp = AgglomerativeClustering(n_clusters=agg_n, linkage='ward').fit(X_dr)
  # To increase efficiency, we calculate the average score of each combination instead.
  tmp_s = 0
  tmp_s += homogeneity_score(y_data, agg_tmp.labels_)
  tmp_s += completeness_score(y_data, agg_tmp.labels_)
  tmp_s += v_measure_score(y_data, agg_tmp.labels_)
  tmp_s += adjusted_rand_score(y_data, agg_tmp.labels_)
  tmp_s += adjusted_mutual_info_score(y_data, agg_tmp.labels_)
  tmp_s /= 5
  agg_scores[0][j] = tmp_s
  print("Agglomerative Clustering when n_clusters =", agg_n, ", average score:", tmp_s)

In [ ]:
mcss = [100, 200]
hdbs_scores = [ [0] * len(DRs) for i in range(2)]
for i in range(len(mcss)):
  print("\nmin_cluster_size =", mcss[i], "...")
  for j in range(len(DRs)):
    print("Current dr:", DRs[j])
    # reading Dimensionality Reduction data
    X_dr = pkl_read(path, DRs[j])
    y_pred_hdbs = hdbscan.HDBSCAN(min_cluster_size=mcss[i], allow_single_cluster=True).fit_predict(X_dr)
    # To increase efficiency, we calculate the average score of each combination instead.
    tmp_s = 0
    tmp_s += homogeneity_score(y_data, y_pred_hdbs)
    tmp_s += completeness_score(y_data, y_pred_hdbs)
    tmp_s += v_measure_score(y_data, y_pred_hdbs)
    tmp_s += adjusted_rand_score(y_data, y_pred_hdbs)
    tmp_s += adjusted_mutual_info_score(y_data, y_pred_hdbs)
    tmp_s /= 5
    hdbs_scores[i][j] = tmp_s
    print("HDBSCAN when min_cluster_size =", mcss[i], ", average score:", tmp_s)

- It takes too long to run the gridsearch to test all possible combinations of the table. We leave the code here in case we want to utilize it in the future.

In [ ]:
# cachedir = mkdtemp()
# memory = Memory(cachedir, verbose=87)
# estimators = [
#     ('vect', CountVectorizer(stop_words='english', min_df=3)),
#     ('tfidf', TfidfTransformer()),
#     ('reduce_dim', None),
#     ('clf', None),
#     ]
# pipeline = Pipeline(estimators, memory=memory)
# param_grid = [{
#         'reduce_dim': [TruncatedSVD(n_components=5, random_state=0),
#                        TruncatedSVD(n_components=20, random_state=0),
#                        TruncatedSVD(n_components=200, random_state=0), 
#                        NMF(n_components=5, init='random', random_state=0),
#                        NMF(n_components=20, init='random', random_state=0),
#                        NMF(n_components=200, init='random', random_state=0),
#                        umap.UMAP(n_components=5, metric='euclidean'),
#                        umap.UMAP(n_components=20, metric='euclidean'),
#                        umap.UMAP(n_components=200, metric='euclidean')],
#         'clf': [KMeans(n_clusters=10, init='k-means++', max_iter=2000, n_init=50, random_state=0),
#                 KMeans(n_clusters=20, init='k-means++', max_iter=2000, n_init=50, random_state=0),
#                 KMeans(n_clusters=50, init='k-means++', max_iter=2000, n_init=50, random_state=0),
#                 AgglomerativeClustering(n_clusters=20, linkage='ward'),
#                 hdbscan.HDBSCAN(min_cluster_size=100),
#                 hdbscan.HDBSCAN(min_cluster_size=200)]
#     }]
# CV = GridSearchCV(pipeline, cv=5, param_grid=param_grid, scoring='accuracy')
# CV.fit(dataset_all.data, y_a_data)

# for id in range(0,len(CV.cv_results_["rank_test_score"])):
#   if CV.cv_results_["rank_test_score"][id] <= 5:
#     print(CV.cv_results_["rank_test_score"][id])
#     print(CV.cv_results_["params"][id])
#     print(CV.cv_results_["mean_test_score"][id])

Ans:

From the result we obtained, the combination of **UMAP using 'cosine' matrix and n_components setting to 5 plus K-Means using k = 20** works best together for 20-class text data.

It might because of that the categories it need to predict is also 20. On the other hand, UMAP captures the structure quickly and efficiently comparing to other methods. Combining these two, they performs better than other combinations.

## Question 18

Extra credit: If you can find creative ways to further enhance the clustering
performance, report your method and the results you obtain.

Ans:

From the result we obtained, we can see that any clustering method using UMAP as the Dimensionality Reduction method performs much better than other methods.

From this perspective, we try to adjust the parameters of UMAP to see if we can obtain a better result.

In [ ]:
rs18 = [30, 50, 100]
for i in range(len(rs18)):
  print("\ntry r =", rs18[i], "...")

  # UMAP
  print("using UMAP ...")
  umap_tmp = umap.UMAP(n_components=rs18[i], metric='cosine')
  X_umap = umap_tmp.fit_transform(X_tfidf)
  pkl_save(path, X_umap, 'umap_' + str(rs18[i]) + '.pkl')

In [ ]:
umaps = ['umap_5.pkl', 'umap_20.pkl', 'umap_30.pkl', 'umap_50.pkl', 'umap_100.pkl', 'umap_200.pkl']

In [ ]:
ks = [10, 20, 30, 50, 100]
km_scores = [ [0] * len(umaps) for i in range(len(ks))]
for i in range(len(ks)):
  print("\nk =", ks[i], "...")
  kmeans_tmp = KMeans(n_clusters=ks[i], init='k-means++', max_iter=2000, n_init=50, random_state=0)
  for j in range(len(umaps)):
    print("Current UMAP:", umaps[j])
    # reading Dimensionality Reduction data
    X_dr = pkl_read(path, umaps[j])
    kmeans_fit = kmeans_tmp.fit(X_dr)
    # To increase efficiency, we calculate the average score of each combination instead.
    tmp_s = 0
    tmp_s += homogeneity_score(y_data, kmeans_fit.labels_)
    tmp_s += completeness_score(y_data, kmeans_fit.labels_)
    tmp_s += v_measure_score(y_data, kmeans_fit.labels_)
    tmp_s += adjusted_rand_score(y_data, kmeans_fit.labels_)
    tmp_s += adjusted_mutual_info_score(y_data, kmeans_fit.labels_)
    tmp_s /= 5
    km_scores[i][j] = tmp_s
    print("Kmeans when k =", ks[i], ", average score:", tmp_s)

From above result, we find that the performace of UMAP_100 is a little bit better than UMAP_50. And $k=20$ is the best setting for K-Means. We will try to adjust some parameters for these two settings to get a better result.

In [ ]:
n_neighs = [20, 30, 50, 100]
min_dists = [0.2, 0.3, 0.5, 0.87]
kmeans_tmp = KMeans(n_clusters=20, init='k-means++', max_iter=2000, n_init=50, random_state=0)
for i in range(len(n_neighs)):
    print("\nn_neighbors =", n_neighs[i], "...")
    for j in range(len(min_dists)):
        print("Current min_dist:", min_dists[j])
        umap_tmp = umap.UMAP(n_components=100, n_neighbors=n_neighs[i], min_dist=min_dists[j], metric='cosine')
        X_umap = umap_tmp.fit_transform(X_tfidf)
        kmeans_fit = kmeans_tmp.fit(X_umap)
        tmp_s = 0
        tmp_s += homogeneity_score(y_data, kmeans_fit.labels_)
        tmp_s += completeness_score(y_data, kmeans_fit.labels_)
        tmp_s += v_measure_score(y_data, kmeans_fit.labels_)
        tmp_s += adjusted_rand_score(y_data, kmeans_fit.labels_)
        tmp_s += adjusted_mutual_info_score(y_data, kmeans_fit.labels_)
        tmp_s /= 5
        km_scores[i][j] = tmp_s
        print("Kmeans when k = 20", ", UMAP when n_neighbors =", n_neighs[i], ", min_dist =", min_dists[j], ", average score:", tmp_s)

After the experiment, we found that the performance usually gets better when we increase the **min_dist** parameter. Which makes sense because as the minimum distance between each data point increase, we should separate the cluster more easily and get a higher Homogeneity and Completeness score.

In conclusion, we found a better clustering performance by using **UMAP with $ n\_components = 100, n\_neighbors = 50$, and $min\_dist = 0.87$. Combining it with K-means with k = 20**, we get a average score which is approximately $0.597$.

## Question 19

In a brief paragraph discuss: If the VGG network is trained on a dataset with
perhaps totally different classes as targets, why would one expect the features derived from such a network to have discriminative power for a custom dataset?

Ans:

We expect that the model is trained on a large and general enough dataset. Even though it's a dataset with perhaps totally different classes as targets, we can still get advantage from its learned feature maps to repurpose it. 

To be specifically, there are always multiple layers inside our network model. We can imagine that the final layer of our network is the one that learn to identify classes specific to our project that need training. The initial layers or the middle layers are used to detect slant lines no matter what you want to classify. Therefore, we can always reuse the neural network instead of retraining them every time we create a new one.

Finally, after getting the pretrained model, we can use fine-tuning to make the model more relevant to the custom dataset we want to use. By doing this, we can expect that the features derived from previous network will have discriminative power for a custom dataset.

## Question 20

In a brief paragraph explain how the helper code base is performing feature
extraction.

Ans:

It first extracts feature layers and pooling layer from the VGG-16. After that, it constructs a flatten layer using torch, and extracts the first part of fully-connected layer from VGG-16.

Similar to the concept we mentioned in Q19, it extracts the basic layers from the pretrained model VGG-16. By doing this, it extracts the basic discriminative pwoer we have in the pretrained model. We can use it in further application such as fine-tuning on the custom dataset we want to test.

## Question 21

In [ ]:
filename = './flowers_features_and_labels.npz'

if os.path.exists(filename):
    file = np.load(filename)
    f_all, y_all = file['f_all'], file['y_all']

else:
    if not os.path.exists('./flower_photos'):
        # download the flowers dataset and extract its images
        url = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
        with open('./flower_photos.tgz', 'wb') as file:
            file.write(requests.get(url).content)
        with tarfile.open('./flower_photos.tgz') as file:
            file.extractall('./')
        os.remove('./flower_photos.tgz')

    class FeatureExtractor(nn.Module):
        def __init__(self):
            super().__init__()

            vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

            # Extract VGG-16 Feature Layers
            self.features = list(vgg.features)
            self.features = nn.Sequential(*self.features)
            # Extract VGG-16 Average Pooling Layer
            self.pooling = vgg.avgpool
            # Convert the image into one-dimensional vector
            self.flatten = nn.Flatten()
            # Extract the first part of fully-connected layer from VGG16
            self.fc = vgg.classifier[0]

        def forward(self, x):
            # It will take the input 'x' until it returns the feature vector called 'out'
            out = self.features(x)
            out = self.pooling(out)
            out = self.flatten(out)
            out = self.fc(out) 
            return out 

    # Initialize the model
    assert torch.cuda.is_available()
    feature_extractor = FeatureExtractor().cuda().eval()

    dataset = datasets.ImageFolder(root='./flower_photos',
                                   transform=transforms.Compose([transforms.Resize(224),
                                                                 transforms.CenterCrop(224),
                                                                 transforms.ToTensor(),
                                                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    # Extract features and store them on disk
    f_all, y_all = np.zeros((0, 4096)), np.zeros((0,))
    for x, y in tqdm(dataloader):
        with torch.no_grad():
            f_all = np.vstack([f_all, feature_extractor(x.cuda()).cpu()])
            y_all = np.concatenate([y_all, y])
    np.savez(filename, f_all=f_all, y_all=y_all)

How many pixels are there in the original images? How many features does
the VGG network extract per image; i.e what is the dimension of each feature vector for an image sample?

Ans:

There are 224 x 224 pixels in the original images.

There are totally 3670 features being extracted by the VGG network per image.

In [ ]:
for x, y in tqdm(dataloader):
  print("\n")
  print(x.size())
  print(y.size())
  break

print("\nfeature vector:", f_all.shape, y_all.shape)
num_features = f_all.shape[1]

## Question 22

Are the extracted features dense or sparse? (Compare with sparse TF-IDF
features in text.)

Ans:

From Q10, we found the shape of TFIDX dataset: (18846, 45365)

The extracted feature vectors we found: (3670, 4096)

Furthermore, we can see that the numbers inside TF-IDF features are much closer to zero comparing to the extracted features. Therefore, the extracted features are more dense.

In [ ]:
print(f_all.shape)
print(f_all)
print(X_tfidf.shape)
print(X_tfidf)

## Question 23

In order to inspect the high-dimensional features, t-SNE is a popular off-the-shelf choice for visualizing Vision features. Map the features you have extracted onto 2 dimensions with t-SNE. Then plot the mapped feature vectors along $x$ and $y$ axes. Color-code the data points with ground-truth labels. Describe your observation.

Ans:

From our result, we can see that there are 5 categories (clusterings) and they have many overlapping spots when we project it onto 2 dimensions.

It indicates that we might face some problems when we try to use high dimensional clustering method on this dataset. The distance between each data point is really close according to our projection result.

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=0)
z = tsne.fit_transform(f_all)
df = pd.DataFrame()
df["y"] = y_all
df["dim_1"] = z[:,0]
df["dim_2"] = z[:,1]
sns.scatterplot(x="dim_1", y="dim_2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", as_cmap = True),
                data=df).set(title="Extracted features maapped onto 2-D with T-SNE projection")

## Question 24

### Autoencoder

In [ ]:
class Autoencoder(torch.nn.Module, TransformerMixin):
    def __init__(self, n_components):
        super().__init__()
        self.n_components = n_components
        self.n_features = None  # to be determined with data
        self.encoder = None
        self.decoder = None
        
    def _create_encoder(self):
        return nn.Sequential(
            nn.Linear(4096, 1280),
            nn.ReLU(True),
            nn.Linear(1280, 640),
            nn.ReLU(True), nn.Linear(640, 120), nn.ReLU(True), nn.Linear(120, self.n_components))
    
    def _create_decoder(self):
        return nn.Sequential(
            nn.Linear(self.n_components, 120),
            nn.ReLU(True),
            nn.Linear(120, 640),
            nn.ReLU(True),
            nn.Linear(640, 1280),
            nn.ReLU(True), nn.Linear(1280, 4096))
    
    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded
    
    def fit(self, X):
        X = torch.tensor(X, dtype=torch.float32, device='cuda')
        self.n_features = X.shape[1]
        self.encoder = self._create_encoder()
        self.decoder = self._create_decoder()
        self.cuda()
        self.train()
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)

        dataset = TensorDataset(X)
        dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in tqdm(range(100)):
            for (X_,) in dataloader:
                X_ = X_.cuda()
                # ===================forward=====================
                output = self(X_)
                loss = criterion(output, X_)
                # ===================backward====================
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        return self     
        
    def transform(self, X):
        X = torch.tensor(X, dtype=torch.float32, device='cuda')
        self.eval()
        with torch.no_grad():
            return self.encoder(X).cpu().numpy()

Report the best result (in terms of rand score) within the table below.
For HDBSCAN, introduce a conservative parameter grid over min cluster size and min samples.

|Module|Alternatives|Hyperparameters|
|---|---|---|
|Dimensionality Reduction|None|N/A|
|Dimensionality Reduction|SVD|r = 50|
|Dimensionality Reduction|UMAP|n_components = 50|
|Dimensionality Reduction|Autoencoder|num_features = 50|
|Clustering|K-Means|k = 5|
|Clustering|Agglomerative Clustering|n_clusters = 5|
|Clustering|HDBSCAN|min_cluster_size & min_samples|


Ans:

The best rand score we get is approximately 0.467. Which is derived from the combination of UMAP [n_components = 50] + K-Means [k = 5].

For HDBSCAN, we set min_cluster_size = [50, 100, 200], min_samples = [20, 50, 100, 200] to proceed a grid search. However, the best rand score we can get is approximately only 0.094 when combining with UMAP [n_components = 50]. Which is still much worse than other combination.

In [ ]:
# modify this line to your local path
pwd = !pwd
path = str(pwd[0]) + '/pickle file/'

In [ ]:
# None
pkl_save(path, f_all, 'f_none.pkl')

r24 = 50
print("\ntry r =", r24, "...")
# SVD
print("using SVD ...")
svd_tmp = TruncatedSVD(n_components=r24, random_state=0)
X_svd = svd_tmp.fit_transform(f_all)
pkl_save(path, X_svd, 'f_svd_' + str(r24) + '.pkl')

# UMAP
print("using UMAP ...")
umap_tmp = umap.UMAP(n_components=r24, metric='cosine')
X_umap = umap_tmp.fit_transform(f_all)
pkl_save(path, X_umap, 'f_umap_' + str(r24) + '.pkl')

# Autoencoder
print("using Autoencoder ...")
X_aec = Autoencoder(r24).fit_transform(f_all)
pkl_save(path, X_aec, 'f_aec_' + str(r24) + '.pkl')


In [ ]:
DRs = ['f_none.pkl', 'f_svd_50.pkl', 'f_umap_50.pkl', 'f_aec_50.pkl']

In [ ]:
ks = [5]
km_scores24 = [ [0] * len(DRs) for i in range(len(ks))]
for i in range(len(ks)):
  print("\nk =", ks[i], "...")
  kmeans_tmp = KMeans(n_clusters=ks[i], init='k-means++', max_iter=2000, n_init=50, random_state=0)
  for j in range(len(DRs)):
    print("Current dr:", DRs[j])
    # reading Dimensionality Reduction data
    X_dr = pkl_read(path, DRs[j])
    kmeans_fit = kmeans_tmp.fit(X_dr)
    # As the question mentioned, we use rand score to determine which combination performs best.
    tmp_s = adjusted_rand_score(y_all, kmeans_fit.labels_)
    km_scores24[i][j] = tmp_s
    print("Kmeans when k =", ks[i], ", rand score:", tmp_s)

In [ ]:
agg_n = 5
agg_scores24 = [ [0] * len(DRs)]
for j in range(len(DRs)):
  print("Current dr:", DRs[j])
  # reading Dimensionality Reduction data
  X_dr = pkl_read(path, DRs[j])
  agg_tmp = AgglomerativeClustering(n_clusters=agg_n, linkage='ward').fit(X_dr)
  # As the question mentioned, we use rand score to determine which combination performs best.
  tmp_s = adjusted_rand_score(y_all, agg_tmp.labels_)
  agg_scores24[0][j] = tmp_s
  print("Agglomerative Clustering when n_clusters =", agg_n, ", rand score:", tmp_s)

In [ ]:
mc_ss = [50, 100, 200]
min_ss = [20, 50, 100, 200]
# hdbs_scores24 = [ [0] * len(DRs) for i in range(2)]
for i in range(len(mc_ss)):
  print("\nmin_cluster_size =", mc_ss[i], "...")
  for k in range(len(min_ss)):
    print("\nmin_samples =", min_ss[k], "...")
    for j in range(len(DRs)):
      print("Current dr:", DRs[j])
      # reading Dimensionality Reduction data
      X_dr = pkl_read(path, DRs[j])
      y_pred_hdbs = hdbscan.HDBSCAN(min_cluster_size=mc_ss[i], min_samples=min_ss[k], allow_single_cluster=True).fit_predict(X_dr)
      # As the question mentioned, we use rand score to determine which combination performs best.
      tmp_s = adjusted_rand_score(y_all, y_pred_hdbs)
      # hdbs_scores24[i][j] = tmp_s
      print("HDBSCAN when min_cluster_size =", mc_ss[i], ", min_samples =", min_ss[k], ", rand score:", tmp_s)

## Question 25

### MLP classifier

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 1280),
            nn.ReLU(True),
            nn.Linear(1280, 640),
            nn.ReLU(True), 
            nn.Linear(640, 5),
            nn.LogSoftmax(dim=1)
        )
        self.cuda()
    
    
    def forward(self, X):
        return self.model(X)
    
    def train(self, X, y):
        X = torch.tensor(X, dtype=torch.float32, device='cuda')
        y = torch.tensor(y, dtype=torch.int64, device='cuda')

        self.model.train()
        
        criterion = nn.NLLLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)

        dataset = TensorDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in tqdm(range(100)):
            for (X_, y_) in dataloader:
                X_ = X_.cuda()
                y_ = y_.cuda()
                # ===================forward=====================
                output = self(X_)
                loss = criterion(output, y_)
                # ===================backward====================
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        return self
    
    def eval(self, X_test, y_test):
        X = torch.tensor(X_test, dtype=torch.float32, device='cuda')
        y = torch.tensor(y_test, dtype=torch.int64, device='cuda')
        dataset = TensorDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=128, shuffle = False)
        correct_pred, num_examples = 0, 0
        for (X_, y_) in dataloader:
            X_ = X_.cuda()
            y_ = y_.cuda()
            logits = self(X_)
            predicted_labels = torch.argmax(logits, 1)
            num_examples += y_.size(0)
            correct_pred += (predicted_labels == y_).sum()
            # print("predicted_labels", predicted_labels)
            # print("y", y_)
        acc = correct_pred.float() / num_examples * 100
        return acc


Report the test accuracy of the MLP classifier on the original VGG features.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(f_all, y_all, test_size=0.2, random_state=0)
print(X_train.shape)

In [ ]:
model = MLP(X_train.shape[1])
model.train(X_train, y_train)
acc = model.eval(X_test, y_test)
print("\nMLP classifier on the original VGG features")
print("Test accuracy: %.2f" % acc, "%")

Report the same when using the reduced-dimension features (you have freedom in choosing the dimensionality reduction algorithm and its parameters).

In [ ]:
umap_50 = umap.UMAP(n_components=50, metric='cosine')
X_umap = umap_50.fit_transform(f_all)
# X_umap = pkl_read(path, 'f_umap_50.pkl')
X_train, X_test, y_train, y_test = train_test_split(X_umap, y_all, test_size=0.2, random_state=0)
print(X_train.shape)

In [ ]:
model = MLP(X_train.shape[1])
model.train(X_train, y_train)
acc = model.eval(X_test, y_test)
print("\nMLP classifier on the reduced-dimension features")
print("Test accuracy: %.2f" % acc, "%")

Does the performance of the model suffer with the reduced-dimension representations? Is it significant?

Ans:

Yes, the performance of the model gets worse when we use the reduced-dimension representation. It is not significant, it only decreases the test accuracy by 4% approximately.

Does the success in classification make sense in the context of the clustering results obtained for the same features in Question 24.

Ans:

Yes, it makes sense. The performance of the model should not be affected a lot if it's already a well-clustering dataset. Even though we reduce the dimensions of the features, the model can still get the structure of the data and performs well eventually.